In [1]:
!pip install kafka-python minio

In [2]:
import pandas as pd
import random
import json
from kafka import KafkaProducer
from minio import Minio
from io import BytesIO
import time

## Configurar cliente MinIO

In [3]:
minio_client = Minio(
    "minio:9000",
    access_key="4PRJYFLGzQYTnOJGH1gA",
    secret_key="ovBkCsqh2cXNkyoteCzQMV5JWCUk5tHfsG1GwYbD",
    secure=False
)

## Carregar dados de clientes

In [4]:
customer_objects = minio_client.list_objects("raw", prefix="pagila_db/customer/", recursive=True)
customer_list = []
for obj in customer_objects:
    data = minio_client.get_object("raw", obj.object_name)
    df = pd.read_parquet(BytesIO(data.read()))
    customer_list.append(df)
customers = pd.concat(customer_list, ignore_index=True)
customers['name'] = customers['first_name'] + ' ' + customers['last_name']
customers['name'].head(10)


0          MARY SMITH
1    PATRICIA JOHNSON
2      LINDA WILLIAMS
3       BARBARA JONES
4     ELIZABETH BROWN
5      JENNIFER DAVIS
6        MARIA MILLER
7        SUSAN WILSON
8      MARGARET MOORE
9      DOROTHY TAYLOR
Name: name, dtype: object

## Carregar dados de filmes

In [5]:
film_objects = minio_client.list_objects("raw", prefix="pagila_db/film/", recursive=True)
films_list = []
for obj in film_objects:
    data = minio_client.get_object("raw", obj.object_name)
    df = pd.read_parquet(BytesIO(data.read()))
    films_list.append(df)
films = pd.concat(films_list, ignore_index=True)
films['title'].head(10)

0    ACADEMY DINOSAUR
1      ACE GOLDFINGER
2    ADAPTATION HOLES
3    AFFAIR PREJUDICE
4         AFRICAN EGG
5        AGENT TRUMAN
6     AIRPLANE SIERRA
7     AIRPORT POLLOCK
8       ALABAMA DEVIL
9    ALADDIN CALENDAR
Name: title, dtype: object

## Montar lista de reviews fictícias

In [6]:
# Com base em sentimentos positivos, neutros e negativos
positive_review = [
    "Um filme incrível! Recomendo muito.",
    "Adorei a atuação e o roteiro.",
    "Excelente produção, vale a pena assistir.",
    "Uma experiência cinematográfica fantástica.",
    "Muito divertido e envolvente!"
]
neutral_review = [
    "O filme é bom, mas nada de especial.",
    "Entretenimento razoável.",
    "Achei interessante, mas poderia ser melhor.",
    "Um filme mediano.",
    "Cumpre o que promete."
]
negative_review = [
    "Não gostei muito do filme.",
    "Achei a história fraca.",
    "Esperava mais dos atores.",
    "O filme é cansativo.",
    "Não recomendo."
]

## Função para gerar uma review aleatória

In [7]:
review = {
    'text': None,
    'rating': None
}

def generate_review():
    tipo = random.choices(
        ['positiva', 'neutra', 'negativa'],
        weights=[0.3, 0.5, 0.2]
    )[0]
    if tipo == 'positiva':
        review['text'] = random.choice(positive_review)
        review['rating'] = random.randint(7, 10)
        return review
    elif tipo == 'neutra':
        review['text'] = random.choice(neutral_review)
        review['rating'] = random.randint(5, 6)
        return review
    else:
        review['text'] =  random.choice(negative_review)
        review['rating'] = random.randint(0, 4)
        return review

## Configurar Kafka Producer

In [8]:
producer = KafkaProducer(
    bootstrap_servers=['kafka:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    #acks='0'  # Não esperar confirmação do broker
)

producer.bootstrap_connected()

True

## Enviar mensagens fictícias a cada 30 segundos

In [ ]:
for _ in range(1000):
    customer = customers.sample(1).iloc[0]
    film = films.sample(1).iloc[0]
    review = generate_review()
    payload = {
        'name': customer['name'],
        'film': film['title'],
        'rating': review['rating'],
        'review': review['text'],
    }
    producer.send(
        topic='popular_critics',
        value=payload
    )
    time.sleep(10)

## Limpando e encerrando canal

In [ ]:
producer.flush()
producer.close()